In [ ]:
import os
import pandas as pd
import sys
sys.path.insert(0, os.path.abspath('../src'))
import config
from evaluate import get_results, evaluate_model_timestamp
import numpy as np

# Troubleshooting and visualisation
import IPython.display as ipd

from tensorflow import keras
from keras_util import train_model, load_model, evaluate_model

# Features
from feat_util import get_train_dev_from_df, get_dev_from_df

# Evaluation with PSDS
from eval import do_eval 

# ComParE Task C: Mosquito Event Detection (MED)
This code is complementary to the paper: [HumBugDB: a large-scale acoustic mosquito dataset](https://arxiv.org/pdf/2110.07607.pdf). Section B of `/docs/` gives detail on the meaning of the metadata fields that are present in the `csv` file `config.data_df`, while Section C describes in more detail the models used here as baselines.


This notebook provides the interface to partition data, extract features, train a BNN model in Keras/TF2.0 and evaluate its accuracy, precision-recall, confusion matrices and uncertainty metrics. Settings are specified in `config.py` and  `config_keras.py` which are located in `../src`. Functions are imported from data and feature processing code in `../src/feat_util.py`, model training in `../src/keras_util.py` and evaluation in `../src/evaluate.py`.

### Data and feature configuration `config.py`
Specify the metadata (csv) location in `data_df`, with the location of the raw wave files in `data_dir`. The desired output for the features is set by `dir_out`. Model objects will be saved to `../models/keras/` for Keras models.

### Features
The baseline uses log-mel features with `librosa`,  based on prior acoustic mosquito detection work [ECML-PKDD2021](https://github.com/HumBug-Mosquito/MozzBNN). The features are configurable in `config.py` with the sample rate `rate`, to which data is re-sampled on loading, a window size `win_size` which determines the size of a training window (in number of _feature_ windows), `step_size`, which determines the step size taken by the window, `NFFT`, and `n_hop`, which are parameters for the core STFT transform upon which log-mel feature extraction is based. Finally, `n_feat` determines the number of mel-frequency bands.

In `librosa`, we can calculate the value of `win_size` to achieve a user's desired `duration` for a label as follows:

`win_size` = `duration` / `frame_duration`, where `frame_duration` = `n_hop`/`rate`. Librosa uses a default `hop_length` of `NFFT/4`.
The default values in `config.py` are optimised for `rate` = 8000 with  `win_size` = 30, `NFFT` = 2048, `n_hop` = `default`,  to achieve a label duration of $30 \times 2048/(4\times 8000) = 1.92$ (s).


### Keras `config_keras.py`
`tau = 1.0`,`lengthscale = 0.01`, are parameters used for $l2$ weight regularization in `keras_util.py`. `dropout = 0.2` controls the dropout rate,`validation_split = 0.2`, is the fraction of data supplied as validation to the model callbacks in `model.fit`. `batch_size` controls the batch size, and `epochs`, set the number of epochs to train. Note the slight difference between the two packages in the way validation data is passed to the model training.

## Step 1a: Data partitioning for feature extraction
Feel free to adjust the training data according to any criteria available from the metadata of the labels, which are stored in `config.data_df`. The code here imports all data that is not in the dev sets to the training set (which can then be further split into validation).
Take extra care to make sure recordings from the same experimental group, as given in Table 2 of the main paper, and indicated in `df['country'], df['location_type']`, do not appear both in train and dev, resulting in overestimate of performance on evaluation. The assertion is given to perform a check to ensure no duplicates arise.

In [ ]:
df = pd.read_csv(config.data_df)

# To be kept: please do not edit the dev set: these paths select dev set a, dev set b as described in the paper
idx_dev_a = np.logical_and(df['country'] == 'Tanzania', df['location_type'] == 'field')
idx_dev_b = np.logical_and(df['country'] == 'UK', df['location_type'] == 'culture')
idx_train = np.logical_not(np.logical_or(idx_dev_a, idx_dev_b))
df_dev_a = df[idx_dev_a]
df_dev_b = df[idx_dev_b]


df_train = df[idx_train]

# Modify by addition or sub-sampling of df_train here
# df_train ... 

# Assertion to check that train does NOT appear in dev:
assert len(np.where(pd.concat([df_train,df_dev_a,
                               df_dev_b]).duplicated())[0]) == 0, 'Train dataframe contains overlap with dev A, dev B'

## Step 1b: Creation of validation data ground truth and labels

In [ ]:
meta_df = df_dev_a[['id', 'length']].copy()
meta_df.rename(columns={"id": "filename", "length": "duration"},inplace=True)
meta_df.to_csv('../data/labels/dev/a/meta.csv', sep='\t', index=False)

### dev a

df_A = df_dev_a[df_dev_a.sound_type == 'mosquito']
gt = df_A[['id', 'length']].copy()
gt.rename(columns={"id": "filename", "length": "offset"},inplace=True)
gt["onset"] = 0
gt["event_label"] = 'mosquito'
gt.to_csv('../data/labels/dev/a/gt.csv', sep='\t', index=False)

### dev b
meta_df = df_dev_b[['id', 'length']].copy()
meta_df.rename(columns={"id": "filename", "length": "duration"},inplace=True)
meta_df.to_csv('../data/labels/dev/b/meta.csv', sep='\t', index=False)


df_B = df_dev_b[df_dev_b.sound_type == 'mosquito']
gt = df_B[['id', 'length']].copy()
gt.rename(columns={"id": "filename", "length": "offset"},inplace=True)
gt["onset"] = 0
gt["event_label"] = 'mosquito'
gt.to_csv('../data/labels/dev/b/gt.csv', sep='\t', index=False)


## Step 2: Feature extraction

In [ ]:
X_train, y_train, X_dev_a, y_dev_a, X_dev_b, y_dev_b = get_train_dev_from_df(df_train, df_dev_a, df_dev_b, debug=True)

### Performance optimisations
When creating features from data with mixed sample rates, `librosa.load` uses re-sampling (if specified) This is a really time consuming process, which may be circumvented by re-sampling the data once and storing in a folder before feature extraction.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

## Step 3: Model training

In [ ]:
model = train_model(X_train, y_train, X_val, y_val)

## Step 4: Model evaluation

The final score will be evaluated with the [PSDS](https://github.com/audioanalytic/psds_eval) scoring function, which is a method for extending event-based F scores to calculate area-under-curve for a range of classifier thresholds. These metrics are calculated below:

### Main evaluation metric:

In [ ]:
# Example in Keras
path = '../models/'
model_name = 'Win_30_Stride_10_2022_04_27_18_45_11-e01val_accuracy0.9798.hdf5' # Succesfully trained TF2 model on full data
model = load_model(path + model_name)

In [ ]:
# Evaluate over dev A
evaluate_model_timestamp('.wav','../data/audio/dev/a','../data/predictions/dev/a', 2, model_weights_path=path+model_name)

In [ ]:
do_eval('./data/predictions/dev/a', './data/labels/dev/a', filename = os.path.join(config.plot_dir, model_name + "_dev_a.png"))

In [ ]:
# Evaluate over dev B
evaluate_model_timestamp('.wav','../data/audio/dev/b','../data/predictions/dev/b', 2, model_weights_path=path+model_name)

In [ ]:
do_eval('./data/predictions/dev/b', './data/labels/dev/b', filename = os.path.join(config.plot_dir, model_name + "_dev_b.png"))

### Optional: also visualise additional information
This section is for visualising the `dev` data in the same fashion as `train`, as well as calculate scores over segment-based sections, where each data sample represents one window of the data. Note that here, edges of the recordings which do not fit into full windows are discarded, which causes some slight discrepancy between the timestamp evaluation methods and the segment-based feature evaluations. You may use these to help train and debug models, but be aware that the final score will be calculated using the PSDS method above.

In [ ]:
X_dev_a, y_dev_a, X_dev_b, y_dev_b = get_dev_from_df(df_dev_a, df_dev_b)

Generate BNN samples. Run with `n_samples` = 1 for deterministic NN, `n` >= 30 for BNN. Calculate the predictive entropy (PE), mutual information (MI), and log probabilities. Also plot the ROC curve and confusion matrix. Outputs are saved to `config.plot_dir` with `filename`. The code automatically aggregates features over the appropriate output shape depending on the feature type defined at the start of the notebook.

### Dev A evaluation

In [ ]:
y_preds_all = evaluate_model(model, X_dev_a, y_dev_a, 30)  # Predict directly over feature windows (1.92 s)
PE, MI, log_prob = get_results(y_preds_all, y_dev_a, filename = model_name +'_dev_a')

### Dev B evaluation

In [ ]:
y_preds_all = evaluate_model(model, X_dev_b, y_dev_b, 30)  # Predict directly over feature windows (1.92 s)
PE, MI, log_prob = get_results(y_preds_all, y_dev_b, filename = model_name +'_dev_b')